# CWAE: Consequence-Weighted AI Evaluation
## Validation on BIRD Benchmark

This notebook validates the CWAE framework against the real BIRD Text-to-SQL benchmark.

**What we'll do:**
1. Install CWAE and dependencies
2. Download BIRD benchmark data
3. Load published model predictions (or generate with a simple baseline)
4. Run both standard EX and CWAE evaluation
5. Compare rankings and demonstrate the materiality effect

---

## 1. Setup & Installation

In [ ]:
# Install dependencies
!pip install -q numpy pandas tqdm requests

# Clone the CWAE repository (or upload the files)
!git clone https://github.com/YOUR_USERNAME/cwae.git 2>/dev/null || echo "Using local files"

# If you uploaded the cwae folder directly, uncomment this:
# (Upload cwae folder to Colab files panel first)
import sys
sys.path.insert(0, '/content/cwae')

print("Setup complete!")

In [ ]:
# Alternative: Paste the core CWAE code directly (no git needed)
# This cell contains the minimal CWAE implementation

import os
os.makedirs('/content/cwae_lib', exist_ok=True)

# Write types.py
types_code = '''
from enum import Enum
from dataclasses import dataclass, field
from typing import Optional, List, Dict, Any

class MaterialityTier(Enum):
    CRITICAL = "critical"
    OPERATIONAL = "operational"
    INFORMATIONAL = "informational"

@dataclass
class MaterialityConfig:
    tier: MaterialityTier
    tolerance_threshold: float
    penalty_multiplier: float
    confidence_discount: float
    
    @classmethod
    def default_critical(cls):
        return cls(MaterialityTier.CRITICAL, 0.005, 2.0, 0.7)
    
    @classmethod
    def default_operational(cls):
        return cls(MaterialityTier.OPERATIONAL, 0.05, 1.0, 0.85)
    
    @classmethod
    def default_informational(cls):
        return cls(MaterialityTier.INFORMATIONAL, 0.10, 0.5, 1.0)

@dataclass
class FMSResult:
    score: float
    quantitative_loss: float
    qualitative_loss: float
    materiality_loss: float
    tier: MaterialityTier
    relative_error: float
    within_tolerance: bool
    predicted_value: Any
    gold_value: Any
    metadata: Dict[str, Any] = field(default_factory=dict)
'''

with open('/content/cwae_lib/types.py', 'w') as f:
    f.write(types_code)

# Write fms.py
fms_code = '''
from typing import Any, Optional, Union
from .types import MaterialityTier, MaterialityConfig, FMSResult

class FMS:
    def __init__(self, tier=MaterialityTier.OPERATIONAL, config=None, w_quant=0.7, w_qual=0.3, epsilon=1e-10):
        self.tier = tier
        self.config = config or self._default_config(tier)
        self.w_quant = w_quant
        self.w_qual = w_qual
        self.epsilon = epsilon
    
    def _default_config(self, tier):
        configs = {
            MaterialityTier.CRITICAL: MaterialityConfig.default_critical(),
            MaterialityTier.OPERATIONAL: MaterialityConfig.default_operational(),
            MaterialityTier.INFORMATIONAL: MaterialityConfig.default_informational(),
        }
        return configs[tier]
    
    def compute_relative_error(self, predicted, gold):
        return abs(predicted - gold) / max(abs(gold), self.epsilon)
    
    def compute_quantitative_loss(self, relative_error):
        tau = self.config.tolerance_threshold
        alpha = self.config.penalty_multiplier
        if relative_error <= tau:
            return 0.0
        scaled = alpha * (relative_error - tau) / (1 - tau)
        return min(1.0, scaled)
    
    def score(self, predicted_value, gold_value, predicted_sql=None, gold_sql=None):
        relative_error = self.compute_relative_error(predicted_value, gold_value)
        within_tolerance = relative_error <= self.config.tolerance_threshold
        l_quant = self.compute_quantitative_loss(relative_error)
        l_qual = 0.0  # Simplified - no SQL comparison
        l_mat = self.w_quant * l_quant + self.w_qual * l_qual
        fms_score = 1.0 - l_mat
        
        return FMSResult(
            score=fms_score, quantitative_loss=l_quant, qualitative_loss=l_qual,
            materiality_loss=l_mat, tier=self.tier, relative_error=relative_error,
            within_tolerance=within_tolerance, predicted_value=predicted_value,
            gold_value=gold_value
        )
    
    def score_binary(self, is_correct, predicted_sql=None, gold_sql=None):
        if is_correct:
            return FMSResult(
                score=1.0, quantitative_loss=0.0, qualitative_loss=0.0,
                materiality_loss=0.0, tier=self.tier, relative_error=0.0,
                within_tolerance=True, predicted_value=1, gold_value=1
            )
        else:
            l_quant = min(1.0, self.config.penalty_multiplier)
            l_mat = self.w_quant * l_quant
            return FMSResult(
                score=max(0.0, 1.0 - l_mat), quantitative_loss=l_quant, qualitative_loss=0.0,
                materiality_loss=l_mat, tier=self.tier, relative_error=1.0,
                within_tolerance=False, predicted_value=0, gold_value=1
            )
'''

with open('/content/cwae_lib/fms.py', 'w') as f:
    f.write(fms_code)

# Write classifier.py
classifier_code = '''
import re
from .types import MaterialityTier

class MaterialityClassifier:
    CRITICAL_PATTERNS = [
        r"\\b(revenue|profit|loss|income|expense|ebitda|margin)\\b",
        r"\\b(total|sum|aggregate)\\b.*\\b(revenue|sales|cost|amount)\\b",
        r"\\b(net|gross)\\s*(revenue|profit|income|margin)\\b",
        r"\\b(quarter|q[1-4]|fiscal|annual|yearly)\\b",
        r"\\b(balance|financial|audit|compliance)\\b",
        r"\\b(tax|liability|asset)\\b",
    ]
    
    OPERATIONAL_PATTERNS = [
        r"\\b(daily|weekly|monthly)\\b",
        r"\\b(average|avg|mean)\\b",
        r"\\b(count|number)\\s*(of)?\\s*(orders|customers|transactions)\\b",
        r"\\b(top|best|worst)\\s*\\d*",
        r"\\b(trend|pattern|comparison)\\b",
    ]
    
    def __init__(self):
        self._critical = [re.compile(p, re.IGNORECASE) for p in self.CRITICAL_PATTERNS]
        self._operational = [re.compile(p, re.IGNORECASE) for p in self.OPERATIONAL_PATTERNS]
    
    def classify(self, query, sql=None):
        text = f"{query} {sql or ''}"
        
        critical_matches = sum(1 for p in self._critical if p.search(text))
        operational_matches = sum(1 for p in self._operational if p.search(text))
        
        if critical_matches > 0:
            return MaterialityTier.CRITICAL
        elif operational_matches > 0:
            return MaterialityTier.OPERATIONAL
        else:
            return MaterialityTier.INFORMATIONAL
'''

with open('/content/cwae_lib/classifier.py', 'w') as f:
    f.write(classifier_code)

# Write __init__.py
init_code = '''
from .types import MaterialityTier, MaterialityConfig, FMSResult
from .fms import FMS
from .classifier import MaterialityClassifier
'''

with open('/content/cwae_lib/__init__.py', 'w') as f:
    f.write(init_code)

sys.path.insert(0, '/content')
print("✓ CWAE library created!")

In [ ]:
# Test the installation
from cwae_lib import FMS, MaterialityClassifier, MaterialityTier

classifier = MaterialityClassifier()
tier = classifier.classify("What was Q3 total revenue?")
print(f"✓ Classification works: '{tier.value}'")

fms = FMS(tier=tier)
result = fms.score(10_234_567, 10_500_000)
print(f"✓ FMS scoring works: {result.score:.3f}")
print(f"  (Binary accuracy would be: 0 - WRONG)")

## 2. Download BIRD Benchmark Data

In [ ]:
import requests
import json
import os

# BIRD dev set is available from their GitHub
# https://github.com/AlibabaResearch/DAMO-ConvAI/tree/main/bird

# Option 1: Download from BIRD's official source
BIRD_DEV_URL = "https://raw.githubusercontent.com/AlibabaResearch/DAMO-ConvAI/main/bird/data/dev/dev.json"

print("Downloading BIRD dev set...")
try:
    response = requests.get(BIRD_DEV_URL, timeout=30)
    if response.status_code == 200:
        bird_data = response.json()
        print(f"✓ Downloaded {len(bird_data)} queries from BIRD dev set")
    else:
        print(f"Download failed with status {response.status_code}")
        bird_data = None
except Exception as e:
    print(f"Download failed: {e}")
    bird_data = None

In [ ]:
# Option 2: If download fails, use a representative sample
# This is a subset of BIRD-style queries for demonstration

if bird_data is None:
    print("Using sample dataset for demonstration...")
    
    # Sample queries representing different materiality tiers
    bird_data = [
        # Critical - Financial queries
        {"question": "What is the total revenue for Q3 2023?", "SQL": "SELECT SUM(amount) FROM transactions WHERE quarter = 'Q3'", "db_id": "financial", "difficulty": "simple"},
        {"question": "Calculate the gross profit margin for last year", "SQL": "SELECT (revenue - cost) / revenue FROM annual_report", "db_id": "financial", "difficulty": "moderate"},
        {"question": "What was the net income after tax?", "SQL": "SELECT net_income FROM financials WHERE year = 2023", "db_id": "financial", "difficulty": "simple"},
        {"question": "Show total assets and liabilities", "SQL": "SELECT assets, liabilities FROM balance_sheet", "db_id": "financial", "difficulty": "simple"},
        {"question": "What is the quarterly revenue breakdown?", "SQL": "SELECT quarter, SUM(revenue) FROM sales GROUP BY quarter", "db_id": "financial", "difficulty": "moderate"},
        {"question": "Calculate EBITDA for the fiscal year", "SQL": "SELECT earnings + depreciation + amortization FROM financials", "db_id": "financial", "difficulty": "moderate"},
        {"question": "What is our tax liability this quarter?", "SQL": "SELECT tax_amount FROM quarterly_taxes WHERE q = 'Q4'", "db_id": "financial", "difficulty": "simple"},
        {"question": "Show the profit and loss statement", "SQL": "SELECT * FROM pnl_report WHERE year = 2023", "db_id": "financial", "difficulty": "simple"},
        
        # Operational - Business metrics
        {"question": "How many orders were placed yesterday?", "SQL": "SELECT COUNT(*) FROM orders WHERE date = CURRENT_DATE - 1", "db_id": "retail", "difficulty": "simple"},
        {"question": "What is the average order value this month?", "SQL": "SELECT AVG(total) FROM orders WHERE month = 12", "db_id": "retail", "difficulty": "simple"},
        {"question": "Show top 10 customers by purchase amount", "SQL": "SELECT customer_id, SUM(amount) FROM orders GROUP BY customer_id ORDER BY 2 DESC LIMIT 10", "db_id": "retail", "difficulty": "moderate"},
        {"question": "What is the daily sales trend this week?", "SQL": "SELECT date, SUM(amount) FROM sales WHERE date >= DATE_SUB(NOW(), 7) GROUP BY date", "db_id": "retail", "difficulty": "moderate"},
        {"question": "Count of active customers in each region", "SQL": "SELECT region, COUNT(*) FROM customers WHERE status = 'active' GROUP BY region", "db_id": "retail", "difficulty": "simple"},
        {"question": "What is the monthly transaction count?", "SQL": "SELECT COUNT(*) FROM transactions WHERE MONTH(date) = 12", "db_id": "retail", "difficulty": "simple"},
        {"question": "Show the weekly sales comparison", "SQL": "SELECT week, SUM(sales) FROM weekly_report GROUP BY week", "db_id": "retail", "difficulty": "simple"},
        {"question": "What is the average customer lifetime value?", "SQL": "SELECT AVG(total_spent) FROM customer_ltv", "db_id": "retail", "difficulty": "simple"},
        
        # Informational - Exploration queries
        {"question": "Show me the list of all products", "SQL": "SELECT * FROM products", "db_id": "retail", "difficulty": "simple"},
        {"question": "What categories do we have?", "SQL": "SELECT DISTINCT category FROM products", "db_id": "retail", "difficulty": "simple"},
        {"question": "List all store locations", "SQL": "SELECT name, address FROM stores", "db_id": "retail", "difficulty": "simple"},
        {"question": "Show the database schema", "SQL": "DESCRIBE tables", "db_id": "retail", "difficulty": "simple"},
        {"question": "What columns are in the orders table?", "SQL": "DESCRIBE orders", "db_id": "retail", "difficulty": "simple"},
        {"question": "Preview the first 10 customer records", "SQL": "SELECT * FROM customers LIMIT 10", "db_id": "retail", "difficulty": "simple"},
        {"question": "What are the product names?", "SQL": "SELECT name FROM products", "db_id": "retail", "difficulty": "simple"},
        {"question": "Show sample transaction data", "SQL": "SELECT * FROM transactions LIMIT 5", "db_id": "retail", "difficulty": "simple"},
    ]
    
    print(f"✓ Created sample dataset with {len(bird_data)} queries")

In [ ]:
# Examine the data
import pandas as pd

df = pd.DataFrame(bird_data)
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nSample queries:")
for i, row in df.head(5).iterrows():
    print(f"  {i+1}. {row['question'][:60]}...")

## 3. Classify All Queries by Materiality

In [ ]:
from cwae_lib import MaterialityClassifier, MaterialityTier
from collections import Counter

classifier = MaterialityClassifier()

# Classify each query
classifications = []
for item in bird_data:
    question = item.get('question', '')
    sql = item.get('SQL', item.get('query', ''))
    tier = classifier.classify(question, sql)
    classifications.append({
        'question': question,
        'sql': sql,
        'tier': tier.value,
        'tier_enum': tier
    })

# Count by tier
tier_counts = Counter(c['tier'] for c in classifications)

print("Materiality Distribution:")
print(f"  Critical:      {tier_counts.get('critical', 0):4d} ({tier_counts.get('critical', 0)/len(classifications)*100:.1f}%)")
print(f"  Operational:   {tier_counts.get('operational', 0):4d} ({tier_counts.get('operational', 0)/len(classifications)*100:.1f}%)")
print(f"  Informational: {tier_counts.get('informational', 0):4d} ({tier_counts.get('informational', 0)/len(classifications)*100:.1f}%)")
print(f"  ─────────────────────")
print(f"  Total:         {len(classifications):4d}")

In [ ]:
# Show examples from each tier
print("\n" + "="*60)
print("EXAMPLES BY TIER")
print("="*60)

for tier_name in ['critical', 'operational', 'informational']:
    examples = [c for c in classifications if c['tier'] == tier_name][:3]
    print(f"\n{tier_name.upper()}:")
    for ex in examples:
        print(f"  • {ex['question'][:70]}")

## 4. Simulate Model Predictions

Since we don't have actual model outputs, we'll simulate predictions with realistic error patterns.

**Simulation approach:**
- Most queries are correct
- Some have small numeric errors (within tolerance)
- Some have larger errors (outside tolerance)
- Error rates vary by model

In [ ]:
import random
import numpy as np

random.seed(42)
np.random.seed(42)

def simulate_model_predictions(classifications, model_profile):
    """
    Simulate model predictions with different error profiles.
    
    model_profile: dict with keys:
        - accuracy: base accuracy rate
        - critical_penalty: additional error rate on critical queries
        - error_magnitude: typical error size when wrong
    """
    predictions = []
    
    for c in classifications:
        tier = c['tier_enum']
        
        # Determine if this prediction is "correct" (exact match)
        base_accuracy = model_profile['accuracy']
        if tier == MaterialityTier.CRITICAL:
            accuracy = base_accuracy - model_profile['critical_penalty']
        else:
            accuracy = base_accuracy
        
        is_exact_match = random.random() < accuracy
        
        # Simulate numeric values
        gold_value = random.randint(1000, 1000000)  # Simulated gold value
        
        if is_exact_match:
            predicted_value = gold_value
            relative_error = 0.0
        else:
            # Generate error
            error_pct = np.random.exponential(model_profile['error_magnitude'])
            error_pct = min(error_pct, 0.5)  # Cap at 50%
            direction = random.choice([-1, 1])
            predicted_value = gold_value * (1 + direction * error_pct)
            relative_error = error_pct
        
        predictions.append({
            'question': c['question'],
            'tier': tier,
            'gold_value': gold_value,
            'predicted_value': predicted_value,
            'is_exact_match': is_exact_match,
            'relative_error': relative_error
        })
    
    return predictions


# Define different model profiles
model_profiles = {
    'Model_A': {
        'accuracy': 0.92,         # 92% overall accuracy
        'critical_penalty': 0.15, # But 15% worse on critical
        'error_magnitude': 0.08   # 8% typical error
    },
    'Model_B': {
        'accuracy': 0.88,         # 88% overall accuracy
        'critical_penalty': -0.05, # Actually BETTER on critical (+5%)
        'error_magnitude': 0.03   # Smaller errors when wrong
    },
    'Model_C': {
        'accuracy': 0.90,         # 90% overall accuracy
        'critical_penalty': 0.05, # Slightly worse on critical
        'error_magnitude': 0.10   # Larger errors
    }
}

# Generate predictions for each model
all_predictions = {}
for model_name, profile in model_profiles.items():
    all_predictions[model_name] = simulate_model_predictions(classifications, profile)
    print(f"✓ Generated predictions for {model_name}")

print(f"\nTotal: {len(all_predictions)} models × {len(classifications)} queries")

## 5. Evaluate with Both Standard EX and CWAE FMS

In [ ]:
from cwae_lib import FMS, MaterialityTier

def evaluate_model(predictions):
    """
    Evaluate predictions with both EX and FMS.
    """
    results = {
        'total': len(predictions),
        'ex_correct': 0,
        'fms_scores': [],
        'by_tier': {
            MaterialityTier.CRITICAL: {'count': 0, 'ex_correct': 0, 'fms_sum': 0},
            MaterialityTier.OPERATIONAL: {'count': 0, 'ex_correct': 0, 'fms_sum': 0},
            MaterialityTier.INFORMATIONAL: {'count': 0, 'ex_correct': 0, 'fms_sum': 0},
        }
    }
    
    for pred in predictions:
        tier = pred['tier']
        
        # Standard EX (binary)
        ex = 1 if pred['is_exact_match'] else 0
        results['ex_correct'] += ex
        results['by_tier'][tier]['ex_correct'] += ex
        results['by_tier'][tier]['count'] += 1
        
        # FMS (graduated)
        fms = FMS(tier=tier)
        if pred['gold_value'] != 0:
            fms_result = fms.score(pred['predicted_value'], pred['gold_value'])
        else:
            fms_result = fms.score_binary(pred['is_exact_match'])
        
        results['fms_scores'].append(fms_result.score)
        results['by_tier'][tier]['fms_sum'] += fms_result.score
    
    # Calculate aggregates
    results['ex_accuracy'] = results['ex_correct'] / results['total']
    results['avg_fms'] = sum(results['fms_scores']) / len(results['fms_scores'])
    
    # Calculate ETS (weighted by tier)
    tier_weights = {
        MaterialityTier.CRITICAL: 2.0,
        MaterialityTier.OPERATIONAL: 1.0,
        MaterialityTier.INFORMATIONAL: 0.5
    }
    
    weighted_sum = 0
    weight_total = 0
    for tier, data in results['by_tier'].items():
        if data['count'] > 0:
            weighted_sum += tier_weights[tier] * data['fms_sum']
            weight_total += tier_weights[tier] * data['count']
    
    results['ets'] = weighted_sum / weight_total if weight_total > 0 else 0
    
    # Tier-level metrics
    for tier in MaterialityTier:
        data = results['by_tier'][tier]
        if data['count'] > 0:
            data['ex_accuracy'] = data['ex_correct'] / data['count']
            data['avg_fms'] = data['fms_sum'] / data['count']
        else:
            data['ex_accuracy'] = 0
            data['avg_fms'] = 0
    
    return results


# Evaluate all models
all_results = {}
for model_name, predictions in all_predictions.items():
    all_results[model_name] = evaluate_model(predictions)
    print(f"✓ Evaluated {model_name}")

## 6. Results Comparison

In [ ]:
# Create comparison table
print("\n" + "="*80)
print("RESULTS COMPARISON: Standard EX vs CWAE")
print("="*80)

# Header
print(f"\n{'Model':<12} {'EX (Binary)':<15} {'ETS (CWAE)':<15} {'Critical FMS':<15} {'Δ Rank'}")
print("-"*72)

# Sort by EX
ex_ranking = sorted(all_results.items(), key=lambda x: x[1]['ex_accuracy'], reverse=True)
ex_ranks = {name: i+1 for i, (name, _) in enumerate(ex_ranking)}

# Sort by ETS
ets_ranking = sorted(all_results.items(), key=lambda x: x[1]['ets'], reverse=True)
ets_ranks = {name: i+1 for i, (name, _) in enumerate(ets_ranking)}

# Display
for model_name, results in sorted(all_results.items()):
    ex = results['ex_accuracy']
    ets = results['ets']
    critical_fms = results['by_tier'][MaterialityTier.CRITICAL]['avg_fms']
    
    rank_change = ex_ranks[model_name] - ets_ranks[model_name]
    rank_str = f"+{rank_change}" if rank_change > 0 else str(rank_change)
    
    print(f"{model_name:<12} {ex:>6.1%} (#{ex_ranks[model_name]})    {ets:>6.1%} (#{ets_ranks[model_name]})    {critical_fms:>6.1%}          {rank_str}")

print("-"*72)

In [ ]:
# Detailed tier breakdown
print("\n" + "="*80)
print("DETAILED BREAKDOWN BY MATERIALITY TIER")
print("="*80)

for model_name, results in sorted(all_results.items()):
    print(f"\n{model_name}:")
    print(f"  {'Tier':<15} {'Count':<8} {'EX Acc':<12} {'FMS Avg':<12}")
    print(f"  {'-'*47}")
    
    for tier in [MaterialityTier.CRITICAL, MaterialityTier.OPERATIONAL, MaterialityTier.INFORMATIONAL]:
        data = results['by_tier'][tier]
        print(f"  {tier.value:<15} {data['count']:<8} {data['ex_accuracy']:>6.1%}       {data['avg_fms']:>6.1%}")
    
    print(f"  {'-'*47}")
    print(f"  {'OVERALL':<15} {results['total']:<8} {results['ex_accuracy']:>6.1%}       {results['ets']:>6.1%} (ETS)")

In [ ]:
# Visualization
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

models = list(all_results.keys())
x = range(len(models))

# Plot 1: EX vs ETS
ax1 = axes[0]
ex_scores = [all_results[m]['ex_accuracy'] for m in models]
ets_scores = [all_results[m]['ets'] for m in models]

width = 0.35
ax1.bar([i - width/2 for i in x], ex_scores, width, label='EX (Binary)', color='steelblue')
ax1.bar([i + width/2 for i in x], ets_scores, width, label='ETS (CWAE)', color='darkorange')
ax1.set_ylabel('Score')
ax1.set_title('Overall: EX vs ETS')
ax1.set_xticks(x)
ax1.set_xticklabels(models)
ax1.legend()
ax1.set_ylim(0.7, 1.0)

# Plot 2: Critical Query Performance
ax2 = axes[1]
critical_ex = [all_results[m]['by_tier'][MaterialityTier.CRITICAL]['ex_accuracy'] for m in models]
critical_fms = [all_results[m]['by_tier'][MaterialityTier.CRITICAL]['avg_fms'] for m in models]

ax2.bar([i - width/2 for i in x], critical_ex, width, label='EX', color='steelblue')
ax2.bar([i + width/2 for i in x], critical_fms, width, label='FMS', color='darkorange')
ax2.set_ylabel('Score')
ax2.set_title('Critical Queries Only')
ax2.set_xticks(x)
ax2.set_xticklabels(models)
ax2.legend()
ax2.set_ylim(0.6, 1.0)

# Plot 3: Ranking Comparison
ax3 = axes[2]
ex_rank_values = [ex_ranks[m] for m in models]
ets_rank_values = [ets_ranks[m] for m in models]

ax3.bar([i - width/2 for i in x], ex_rank_values, width, label='EX Rank', color='steelblue')
ax3.bar([i + width/2 for i in x], ets_rank_values, width, label='ETS Rank', color='darkorange')
ax3.set_ylabel('Rank (lower is better)')
ax3.set_title('Ranking: EX vs ETS')
ax3.set_xticks(x)
ax3.set_xticklabels(models)
ax3.legend()
ax3.set_ylim(0, 4)
ax3.invert_yaxis()

plt.tight_layout()
plt.savefig('cwae_comparison.png', dpi=150)
plt.show()

print("\n✓ Chart saved as 'cwae_comparison.png'")

## 7. Key Insights

In [ ]:
print("\n" + "="*80)
print("KEY INSIGHTS")
print("="*80)

# Find ranking changes
ranking_changes = []
for model in models:
    ex_rank = ex_ranks[model]
    ets_rank = ets_ranks[model]
    if ex_rank != ets_rank:
        ranking_changes.append((model, ex_rank, ets_rank))

if ranking_changes:
    print("\n📊 RANKING CHANGES:")
    for model, ex_r, ets_r in ranking_changes:
        direction = "↑" if ets_r < ex_r else "↓"
        print(f"   {model}: #{ex_r} (EX) → #{ets_r} (ETS) {direction}")

# Find the model that looks best by EX but worst on critical
ex_best = ex_ranking[0][0]
critical_scores = [(m, all_results[m]['by_tier'][MaterialityTier.CRITICAL]['avg_fms']) for m in models]
critical_ranking = sorted(critical_scores, key=lambda x: x[1], reverse=True)

print(f"\n⚠️  WARNING:")
print(f"   {ex_best} ranks #1 by standard EX but may not be safest for production!")
print(f"   On CRITICAL queries: {all_results[ex_best]['by_tier'][MaterialityTier.CRITICAL]['avg_fms']:.1%}")
print(f"   Best on critical: {critical_ranking[0][0]} at {critical_ranking[0][1]:.1%}")

print("\n💡 THE POINT:")
print("   Standard accuracy metrics can be misleading for enterprise AI.")
print("   A model optimized for overall accuracy may underperform on")
print("   the queries that matter most to your business.")
print("\n   CWAE surfaces these risks by weighting errors by consequence.")

## 8. Export Results

In [ ]:
# Export to CSV
results_df = pd.DataFrame([
    {
        'model': model,
        'ex_accuracy': results['ex_accuracy'],
        'ets': results['ets'],
        'avg_fms': results['avg_fms'],
        'critical_count': results['by_tier'][MaterialityTier.CRITICAL]['count'],
        'critical_ex': results['by_tier'][MaterialityTier.CRITICAL]['ex_accuracy'],
        'critical_fms': results['by_tier'][MaterialityTier.CRITICAL]['avg_fms'],
        'operational_fms': results['by_tier'][MaterialityTier.OPERATIONAL]['avg_fms'],
        'informational_fms': results['by_tier'][MaterialityTier.INFORMATIONAL]['avg_fms'],
        'ex_rank': ex_ranks[model],
        'ets_rank': ets_ranks[model],
    }
    for model, results in all_results.items()
])

results_df.to_csv('cwae_results.csv', index=False)
print("✓ Results saved to 'cwae_results.csv'")
print("\n")
display(results_df)

---

## Next Steps

To validate with **real data**:

1. **Get BIRD databases**: Download the actual SQLite files from BIRD
2. **Get model predictions**: Use published model outputs or run models yourself
3. **Execute SQL**: Actually run both gold and predicted SQL to get real values
4. **Compare**: See if the ranking shift holds on real data

The framework is ready - you just need real predictions to plug in!